# 2.2.9 Estimación de estadísticos usando bootstrap

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from scipy.stats import norm, lognorm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../../../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Analitica predictiva'
data = make_dir_line(modality, project)

raw = data('raw')

La estimación de la distribución de probabilidades (y sus propiedades) de un estadístico suele ser dificil debido a que solo unos pocos estadísticos siguen una distribución de probabilidades conocida. El bootstrap es un método numérico para realizar está estimación a partir de una muestra de datos. Uno de sus usos más importantes es la estimación del error promedio de pronóstico en modelos predictivos comúnmente usados en analítica.

## Descripción del problema

Se tiene la siguiente muestra de datos, obtenida de un experimento. Se desea estimar la media de la población.

In [4]:
x = [0.09, 0.15, 0.18, 0.16, 0.23, 0.35, 0.49, 0.68, 0.71, 0.85, 0.96, 0.98, 0.27]

## Conceptos básicos

A continuación se presentan las principales definiciones asociadas al concepto de bootstrap.

![Alt text](images/bootstrap_1.png)

**Población:** Colección de objetos o individuos sobre los cuales se desea obtener información de interés.

**Muestra:** Grupo obtenido aleatoriamente que es representativo de la población.

**Parámetro:** Un valor que resume una propiedad o característica de la población entera.

**Muestreo sin reemplazo:** Cuando una observación es seleccionada ya no está disponible en adelante.

**Muestreo con reemplazo:** Cada observación es devuelta a la población después de ser seleccionada.

**Muestreo aleatorio:** Muestreo de elementos de forma aleatoria tal que cada miembro de la población tiene igual probabilidad de ser seleccionado.

**Muestreo estratificado:** División de la población en estratos y muestreo aleatorio de cada estrato.

**Muestra aleatoria simple:** Muestra que resulta de muestrear la población sin estratificar.

**Muestra sesgada:** Muestra que no representa la población.

**Sesgo estadístico:** Errores de muestreo o medición que son sistemáticos y producidos por la medición o el muestreo mismo.

**Estadístico:** Un número que resume una propiedad o característica de la muestra.

**Estadístico muestral:** Una métrica calculada para una muestra de datos obtenida de forma aleatoria de la población.

**Distribución muestral:** Es la distribución de probabilidad de un estadístico muestral calculado sobre muchas muestras o remuestreos.

**Error estándar:** Es la desviación estándar del estadístico muestral sobre muchas muestras.

Para estimar el valor de un parámetro de la población mediante un estadístico calculado sobre la muestra, se deben tener en cuenta alguna de las dos siguientes opciones:

- Pruebas de hipótesis (se caracterizaron en el módulo de análisis exploratorio).
- Estimación de intervalos de confianza.

El problema abordado aquí radica en que el estadístico calculado (la media de la muestra) es una aproximación al parámetro real de la población (la media de la población); si se obtienen diferentes muestras de datos, y se calcula el estadístico de interés, se obtendrán distintos valores (distintas medias); entonces, la pregunta es cómo aproximar la media de la población a partir de la muestra de datos. Esta es la utilidad del bootstrapping.

## Bootstrapping



In [5]:
print('ok_')

ok_
